In [1]:
import pandas as pd
import numpy as np
import os
import re

from IPython.core.display import display, HTML
import html
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import markdown

/tmp/ipykernel_3881416/908490600.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
os.chdir('/shared/0/projects/research-jam-summer-2024/')

In [14]:
inst = "Simulate a person using an LLM. Respond like a regular person. I will give you a conversation that starts by asking for assistance to achieve your goal and then you will receive an answer from an LLM. If you goal achieved, you should not respond. If not, you will write the person's next response based on both previous turns! Generate only the last human response."
def convert_to_html(text):
    text = html.escape(text)
    text = re.sub('\n','<br/>',text)
    text = re.sub('\t','&emsp;',text)
    return text

def render_block_style(text, background_color="#f9f9f9"):
    return f'''<div style="
    background-color: {background_color};
    border: 1px solid #ddd;
    padding: 10px;
    margin: 10px 0;
    border-radius: 5px;
    line-height: 1.6;">{text}</div>'''

def highlight_non_tag_text(i):
    highlighted_text1 = render_block_style(convert_to_html(sample.human_turn_1[i]), "#e6f7ff")
    #highlighted_text1 = render_block_style(markdown.markdown(sample.human_turn_1[i], extensions=['extra', 'fenced_code', 'nl2br']), "#e6f7ff")
    highlighted_text2 = render_block_style(markdown.markdown(sample.ai_turn_2[i], extensions=['extra', 'fenced_code', 'nl2br']), "#fff3e6")
    return inst + '<br/><br/>' + 'You said this to an LLM: ' + '<br/>' + highlighted_text1 + '<br/><br/>' + 'LLM response:' + '<br/>' + highlighted_text2


In [4]:
sample = pd.read_json('data/english_only/wildchat_subset_en_2k.jsonl', orient='records', lines=True)

In [5]:
sample['id'] = sample['hashed_ip'] + sample['conversation_hash']

In [6]:
sample['no_response'] = (sample.human_turn_3 == '[no response]') | (sample.human_turn_3.apply(lambda x: len(re.sub('\n|\s|\r','',x))==0))
Counter(sample['no_response'])

Counter({True: 1258, False: 742})

In [7]:
nr = sample[sample['no_response']]['id'].sample(frac=0.5, replace=False, random_state=0).tolist()
r = sample[~sample['no_response']]['id'].sample(frac=2, replace=True, random_state=0).tolist()

In [8]:
sample = pd.concat([sample.set_index('id').loc[nr], sample.set_index('id').loc[r]]).reset_index()

In [15]:
#sample['text'] = sample['human_turn_1'].str.cat(sample['ai_turn_2'])
sample['text'] = sample.reset_index()['index'].apply(highlight_non_tag_text)

In [16]:
# response/no response
sample[['id','text']].to_csv('/home/akananth/Misc/human-llm-similarity/src/annotation/data_files/for_human_3_annotation.csv',
                             index=None)

In [20]:
sample = pd.read_csv('/home/akananth/Misc/human-llm-similarity/src/annotation/data_files/for_human_3_annotation.csv')


Index(['649', '677', '687', '704', '716', '717', '733', '750', '759', '761',
       ...
       '2102', '2104', '2105', '2106', '2107', '2108', '2109', '2110', '2111',
       '2112'],
      dtype='object', length=840)

In [37]:
sample['id'][sample['id'].duplicated()] + "_" + sample.index[sample['id'].duplicated()].astype(str)

['a1fe56efdd426695e35aee0d6316c5a623f1631f5cdaa61b31d14be5ddad9524232da82f9cf6d5f4fafeedd18796389d_649',
 'd9c3ef923024d01ad073b79acc040cb3ccc9947edcc5fd544472cd650d66bc91c7786e939f7901d107416594e19e6954_677',
 'ad0f8b22ea20d4e00d3fe5c9a3394aa973db2c8469d0c21d73242dc9e0ff8d3ffe32b996d95b4b58a908c9faacd30e7b_687',
 '2c0483ac16465fcbf265a865a7de90a6ed0d1e33fca47bed0a4f8a46e7b20142e33724141c7d1ad7ba826e2a8bc3984a_704',
 '44dff456a2ae894bf7924fe0272fb652a8e24447ab2123860d4560eb53312c70158329dd68d72b04abba9d6f0f22d8ff_716',
 '3618f332f87a28193510ac02748f7ebe6a7ece470493b15bacedf5cdaf57c496177b56c4d266416ef4a544b477c6fca6_717',
 'dee4329ee7a7ad76a3dc278f2eec1363678bda5a4022782e6bd8b9ba25109cc148dfad2df8eaef04292d6764d4ea0c5a_733',
 '99be0fdada1191e384e30a2eda55e10a6dc47ec1ca1d9878590b194b2d195cbf4d38da243a08272be401eb509a433a7d_750',
 'd9c3ef923024d01ad073b79acc040cb3ccc9947edcc5fd544472cd650d66bc91c7786e939f7901d107416594e19e6954_759',
 '40ad88d85462c815ea4f871251ae5c2feee0dc7701d7e1ec6f024

In [17]:
display(HTML(sample.text[1]))

In [18]:
display(HTML(sample.text[7]))

In [19]:
display(HTML(sample.text[108]))